In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.environ["GROQAI_API_KEY"]=os.getenv("GROQAI_API_KEY")
hf_api = os.getenv("HF_TOKEN")

In [2]:
#from langchain_groq import ChatGroq
#llm=ChatGroq(temperature=0.5,api_key=api_key,model_name="mixtral-8x7b-32768")
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

# Use a RoBERTa model specifically for causal LM (decoder mode)
model_name = "meta-llama/Llama-3.2-1B"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True,max_length=256, padding=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, is_decoder=True,torch_dtype=torch.float16,  # Use float16 for faster inference
    force_download=True)  # Automatically assign model to GPU if available

tokenizer.pad_token = tokenizer.eos_token

# Create a text generation pipeline
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=200,  # Increased from 50
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id  # Explicitly set pad token
)

# Wrap the pipeline using LangChain
llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cpu


In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
from bs4 import SoupStrainer
loader=WebBaseLoader(web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
                     bs_kwargs=dict(
                         parse_only=SoupStrainer(class_=("post-content", "post-title"))
                     ),
                 )
docs=loader.load()
docs


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n      LLM Powered Autonomous Agents\n    Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\

In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=50)
text=text_splitter.split_documents(docs)
text

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Agent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:'),
 Document(metadata=

In [7]:
db=Chroma.from_documents(docs,embeddings)

# Add document filtering and limit results
retriever = db.as_retriever(
    search_kwargs={
        "k": 3,  # Return top 3 most relevant chunks
        "score_threshold": 0.7  # Minimum relevance score
    }
)

In [8]:
system_prompt = """You're an assistant for Q&A tasks. Use ONLY the provided context to answer.
Format your answer as:
- Concise explanation (2-3 sentences)
- Key points as bullet items
- Conclusion sentence

If unsure, say "I don't know."

Context:
{context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Question: {input}")
])

In [9]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever, question_answer_chain)

from langchain_core.output_parsers import StrOutputParser

# Add to your chain construction
cleaned_chain = rag_chain | StrOutputParser()

In [10]:
response = cleaned_chain.invoke({"input": "What is Self-Reflection"})
response

TypeError: Collection.query() got an unexpected keyword argument 'score_threshold'